In [1]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D 
import numpy as np
import PIL
tf.__version__

#dataset - https://drive.google.com/file/d/1iQ3BgBx031F0qSUm-geAzwbNx1GY4gEQ/view

'2.4.1'

In [2]:
training_datagen = ImageDataGenerator(rescale=1./255,
                                      rotation_range=30,
                                      horizontal_flip=True,
                                      vertical_flip=True,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      height_shift_range=0.2,
                                      width_shift_range=0.2,)


validation_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
train = training_datagen.flow_from_directory("./FaceExpressionRecog/train", 
                                            target_size = (48, 48),
                                            color_mode='grayscale',
                                            class_mode='categorical',
                                            batch_size=32,
                                            shuffle=True)
#256x256 is default

val = validation_datagen.flow_from_directory("./FaceExpressionRecog/validation", 
                                            target_size = (48, 48),
                                            color_mode='grayscale',
                                            class_mode='categorical',
                                            batch_size=32)

Found 24282 images belonging to 5 classes.
Found 5937 images belonging to 5 classes.


In [4]:
img_rows, img_cols = 48, 48

model = Sequential()

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [5]:
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [6]:
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [7]:
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [8]:
model.add(Flatten())

model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [9]:
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [10]:
model.add(Dense(5,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [11]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0

In [12]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint('Emotion_little_vgg.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]

In [13]:
model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])

In [14]:
history=model.fit(
                train,
                steps_per_epoch=train.samples//32,
                epochs=25,
                callbacks=callbacks,
                validation_data=val,
                validation_steps=val.samples//32)

Epoch 1/25
758/758 [==============================] - 417s 548ms/step - loss: 2.0794 - accuracy: 0.2287 - val_loss: 1.5741 - val_accuracy: 0.3008

Epoch 00001: val_loss improved from inf to 1.57407, saving model to Emotion_little_vgg.h5
Epoch 2/25
758/758 [==============================] - 419s 553ms/step - loss: 1.5913 - accuracy: 0.2775 - val_loss: 1.5086 - val_accuracy: 0.3296

Epoch 00002: val_loss improved from 1.57407 to 1.50861, saving model to Emotion_little_vgg.h5
Epoch 3/25
758/758 [==============================] - 420s 554ms/step - loss: 1.5229 - accuracy: 0.3198 - val_loss: 1.3879 - val_accuracy: 0.4117

Epoch 00003: val_loss improved from 1.50861 to 1.38789, saving model to Emotion_little_vgg.h5
Epoch 4/25
758/758 [==============================] - 421s 555ms/step - loss: 1.4378 - accuracy: 0.3768 - val_loss: 1.2414 - val_accuracy: 0.4821

Epoch 00004: val_loss improved from 1.38789 to 1.24140, saving model to Emotion_little_vgg.h5
Epoch 5/25
758/758 [====================